# Manipulate data #

In [2]:
import pandas as pd
import pandas_gbq
import re

In [ ]:
df_pthList=pd.read_csv('https://github.com/wichukornk/FEC/raw/main/list_folder2.txt',header=None)
a=[]
for i in df_pthList.loc[:,0]:
    if  re.search('Energy_input.+monthly.xls',i):
        if "P001" in i:
            #if 'aseline' in i:
            b=i.replace('\\','/')
            a.append(b)
        else:
            b=i.replace('\\','/')
            a.append(b)
#print(a)
pth=pd.DataFrame(a)
pth=pth.loc[:,0].str.split(pat='/',expand=True)
pth[11]=""
for i in pth.index:
  if "CHW_P_" in pth.loc[i,10]:
    pth.loc[i,11]="2.1"
  elif "CT_" in pth.loc[i,10]:
    pth.loc[i,11]="2.2"
  elif "CW_P_" in pth.loc[i,10]:
    pth.loc[i,11]="2.3"
    #print(pth.loc[i,10])
  else:
    pth.loc[i,11]=pth.loc[i,5]
#pth

,0,1,2,3,4,5,6,7,8,9,10,11
0,C:,inetpub,wwwroot,trnsysmodel,F00000000000001,1,P006,B1 unit,Test,Datalogger,Energy_input_monthly.xls,1
1,C:,inetpub,wwwroot,trnsysmodel,F00000000000001,1,P006,B1 unit,TestChange3,Datalogger,Energy_input_monthly.xls,1
2,C:,inetpub,wwwroot,trnsysmodel,F00000000000001,1,P006,B3 units,testsave,Datalogger,Energy_input_monthly.xls,1
3,C:,inetpub,wwwroot,trnsysmodel,F00000000000001,1,P006,B3 units,testsave2,Datalogger,Energy_input_monthly.xls,1
4,C:,inetpub,wwwroot,trnsysmodel,F00000000000001,1,P014,B1 unit,Test,Datalogger,Energy_input_monthly.xls,1
...,...,...,...,...,...,...,...,...,...,...,...,...
162,C:,inetpub,wwwroot,trnsysmodel,F00000000000037,3,P008,1 unit Aircomp,Factory Thai,Datalogger,Energy_input_monthly.xls,3
163,C:,inetpub,wwwroot,trnsysmodel,F00000000000037,3,P008,Aircompressor 1 unit,Factory Thai 1,Datalogger,Energy_input_monthly.xls,3
164,C:,inetpub,wwwroot,trnsysmodel,F00000000000038,1,P006,B1 unit,boiler testing,Datalogger,Energy_input_monthly.xls,1
165,C:,inetpub,wwwroot,trnsysmodel,F00000000000038,1,P007,B1 unit,boiler testing,Datalogger,Energy_input_monthly.xls,1


In [ ]:
df_getData=pd.DataFrame()
df_getData['factory']=pth.loc[:,4]
df_getData['system']=pth.loc[:,5]
df_getData['measure']=pth.loc[:,6]
df_getData['model']=pth.loc[:,8]
df_getData['equipment']=pth.loc[:,11]
df_getData['link']="http://54.254.220.213/trnsysmodel"
for i in range(7):
    df_getData['link']=df_getData['link']+"/"+pth.loc[:,i+4].str.replace(' ','%20')
#df_getData.head()

,factory,system,measure,model,equipment,link
0,F00000000000001,1,P006,Test,1,http://54.254.220.213/trnsysmodel/F00000000000...
1,F00000000000001,1,P006,TestChange3,1,http://54.254.220.213/trnsysmodel/F00000000000...
2,F00000000000001,1,P006,testsave,1,http://54.254.220.213/trnsysmodel/F00000000000...
3,F00000000000001,1,P006,testsave2,1,http://54.254.220.213/trnsysmodel/F00000000000...
4,F00000000000001,1,P014,Test,1,http://54.254.220.213/trnsysmodel/F00000000000...


In [ ]:
def colToRow(df_temColData,factory,system,model,equipment,measure):
    df_tod2=pd.DataFrame(columns=['no','Period','value'])
    month=df_temColData.loc[:,'Period']
    for k in range(df_temColData.shape[1]-1):
        temp=pd.DataFrame(month)
        temp['value']=df_temColData.iloc[:,k+1]
        temp['no']=k+1
        df_tod2=df_tod2.append(temp)
    df_tod2['factory']=factory
    df_tod2['system']=system
    df_tod2['model']=model
    df_tod2['equipment']=equipment
    df_tod2['measure']=measure
    return df_tod2.loc[:,['factory','system','measure','model','equipment','no','Period','value']]

df=pd.DataFrame(columns=['factory','system','measure','model','equipment','no','Period','value'])
for i in df_getData.index:
    try:
        df_temColData=pd.read_csv(df_getData.loc[i,'link'],sep='\t',skiprows=1,nrows=12)
        df_temColData=df_temColData.dropna(axis=1)
        df_tod=colToRow(df_temColData,df_getData.loc[i,'factory'],df_getData.loc[i,'system'],df_getData.loc[i,'model'],df_getData.loc[i,'equipment'],df_getData.loc[i,'measure'])
        df=df.append(df_tod)
        #print(df_getData.loc[i,'system'],df_temColData.shape,df.shape)
    except:
        print(df_getData.loc[i,'factory'],df_getData.loc[i,'system'])

In [ ]:
df

,factory,system,measure,model,equipment,no,Period,value
0,F00000000000001,1,P006,Test,1,1,January,3.733150e+09
1,F00000000000001,1,P006,Test,1,1,February,3.370747e+09
2,F00000000000001,1,P006,Test,1,1,March,3.728611e+09
3,F00000000000001,1,P006,Test,1,1,April,3.603879e+09
4,F00000000000001,1,P006,Test,1,1,May,3.719391e+09
...,...,...,...,...,...,...,...,...
7,F00000000000038,3,P009,testing02,3,1,August,3.958368e+06
8,F00000000000038,3,P009,testing02,3,1,September,3.830871e+06
9,F00000000000038,3,P009,testing02,3,1,October,3.959049e+06
10,F00000000000038,3,P009,testing02,3,1,November,3.830514e+06


In [ ]:
df2=df.copy()

In [ ]:
df['system']=df['system'].map({'1':'Boiler','2':'Chiller','3':'Compressor'})
df['equipment']=df['equipment'].map({'1':'Boiler','2':'Chiller','3':'Compressor','2.1':'CHW_P','2.2':'CT','2.3':'CW_P'})
#df['equipment'].value_counts()

In [ ]:
df_factoryID=pd.read_csv("https://github.com/wichukornk/FEC/raw/main/factoryID.txt",header=None)
df_factoryID[0]=df_factoryID[0].str.replace("<option value=\"","")
df_factoryID[0]=df_factoryID[0].str.replace("</option>","")
df_factoryID[0]=df_factoryID[0].str.replace("\">",",")
df_factoryID=df_factoryID[0].str.split(pat=",",expand=True)
df['factory']=df['factory'].map(dict(zip(df_factoryID[0],df_factoryID[1])))

In [ ]:
df_meaID=pd.read_csv("https://github.com/wichukornk/FEC/raw/main/measureID.txt",header=None)
df_meaID[0]=df_meaID[0].str.replace("<option value=\"","")
df_meaID[0]=df_meaID[0].str.replace("</option>","")
df_meaID[0]=df_meaID[0].str.replace("\">",",")
df_meaID=df_meaID[0].str.split(pat=",",expand=True)
df['measure']=df['measure'].map(dict(zip(df_meaID[0],df_meaID[1])))
#df_meaID

In [ ]:
df.dropna(axis=0)

,factory,system,measure,model,equipment,no,Period,value
0,บริษัท บีอินสปายด์พลาสติก จำกัด,Boiler,ระบบหม้อน้ำ Boiler เปลี่ยนหม้อน้ำประสิทธิภาพสูง,Test,Boiler,1,January,3.733150e+09
1,บริษัท บีอินสปายด์พลาสติก จำกัด,Boiler,ระบบหม้อน้ำ Boiler เปลี่ยนหม้อน้ำประสิทธิภาพสูง,Test,Boiler,1,February,3.370747e+09
2,บริษัท บีอินสปายด์พลาสติก จำกัด,Boiler,ระบบหม้อน้ำ Boiler เปลี่ยนหม้อน้ำประสิทธิภาพสูง,Test,Boiler,1,March,3.728611e+09
3,บริษัท บีอินสปายด์พลาสติก จำกัด,Boiler,ระบบหม้อน้ำ Boiler เปลี่ยนหม้อน้ำประสิทธิภาพสูง,Test,Boiler,1,April,3.603879e+09
4,บริษัท บีอินสปายด์พลาสติก จำกัด,Boiler,ระบบหม้อน้ำ Boiler เปลี่ยนหม้อน้ำประสิทธิภาพสูง,Test,Boiler,1,May,3.719391e+09
...,...,...,...,...,...,...,...,...
7,บริษัท ไทยร่วมใจโคราช จำกัด,Compressor,ค่าเริ่มต้น,Thai Ruamjai Korat Aircom Baseline,Compressor,2,August,1.973300e+06
8,บริษัท ไทยร่วมใจโคราช จำกัด,Compressor,ค่าเริ่มต้น,Thai Ruamjai Korat Aircom Baseline,Compressor,2,September,1.910459e+06
9,บริษัท ไทยร่วมใจโคราช จำกัด,Compressor,ค่าเริ่มต้น,Thai Ruamjai Korat Aircom Baseline,Compressor,2,October,1.975169e+06
10,บริษัท ไทยร่วมใจโคราช จำกัด,Compressor,ค่าเริ่มต้น,Thai Ruamjai Korat Aircom Baseline,Compressor,2,November,1.910459e+06


# Update BigQuery #

In [ ]:
df.dropna(axis=0).to_gbq(destination_table='FEC.energy_input_monthly',project_id='introcloud01',if_exists='replace')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=FtvVjOUQmWEhbraWb7WuzbUFkEsfw3&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g7v43e9T4_abhhpACPJDMRPvXppYTLKUPrWQLzi-hvydnc0cO5JwXw


1it [00:02,  2.58s/it]
